##### 인스타 해시태그 크롤링 ##

## 1. 인스타그램 특정 >> 게시글(URL) <<데이터 크롤링

##### [조건] 
#####  1. 로그인 X
#####  2. 특정 게시글(URL) 대상
#####  3. 계정명, 좋아요 수, 해시태그, 본문 벌크, 게시 날짜, 추출 날짜, 결과 csv 저장(계정명.csv)
##### [문제점]
##### 1. 좋아요 수 집계 불가

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from datetime import datetime

# 1. 웹드라이버 설정 및 페이지 이동
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 인스타그램 릴스 URL
url = "https://www.instagram.com/reel/C_IGZmRyUU9/?utm_source=ig_web_copy_link&igsh=MzRlODBiNWFlZA=="
driver.get(url)
time.sleep(10)  # 페이지 로드 대기

# 2. 데이터 추출
# 계정명 추출
try:
    account_name = driver.find_element(By.XPATH, '//a[contains(@class, "notranslate")]').text
except:
    account_name = "N/A"


# 좋아요 수 추출
try:
    # 좋아요 수가 포함된 span 요소를 찾습니다.
    likes_element = driver.find_element(By.XPATH, '//div[contains(text(), "likes")]')
    likes = likes_element.text.split(' ')[0]  # 'likes' 텍스트 앞의 숫자만 추출
except:
    likes = "N/A"
    

# 해시태그 추출
try:
    hashtags = [tag.text for tag in driver.find_elements(By.XPATH, '//a[contains(@href, "/explore/tags/")]')]
    hashtags = hashtags[:10]  # 상위 10개 해시태그만 가져오기
except:
    hashtags = []

# 본문 텍스트 추출
try:
    post_text = driver.find_element(By.XPATH, '//div[@class="_a9zr"]').text
except:
    post_text = ""

# 게시 날짜 추출
try:
    post_date_element = driver.find_element(By.XPATH, '//time')
    post_date = post_date_element.get_attribute('datetime')
    post_date = datetime.strptime(post_date, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")  # ISO 형식을 datetime으로 변환
except:
    post_date = "N/A"

# 현재 크롤링 시간
crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 3. 결과 출력 및 CSV 저장
data = {
    'account_name': account_name,
    'likes': likes,
    'hashtags': ', '.join(hashtags),
    'post_text': post_text,
    'post_date': post_date,
    'crawl_time': crawl_time,
    'post_url': url
}

# CSV 저장
df = pd.DataFrame([data])
df.to_csv('{}.csv'.format(account_name), index=False)

print(f"Data saved to instagram_post_data.csv: {data}")


# 드라이버 종료
driver.quit()

Data saved to instagram_post_data.csv: {'account_name': '@sh.cookkook', 'likes': 'N/A', 'hashtags': '#입분식가정집, #송파맛집, #송리단길, #송리단길맛집, #잠실맛집, #즉석떡볶이, #즉떡', 'post_text': 'sh.cookkook\n줄서는식당에도 나온 즉떡?\n\n어릴 적부터 주구장창 먹고자란 저란 아이..\n여기 즉떡은 육수+특제 소스로 찐한맛이라\n자극적인 맛 러버들이 특히 좋아할 곳>.<\n\n그리고 우삼겹이나 대창 추가도 할 수 있어\n특별하게 느껴졌고, 저는 이번엔 계란이랑\n어떡만 추가했어요!(어묵 안에 떡이 있음)\n\n기본으로만 먹어도 양 충분했고 넘모넘모\n맛있음ㅜ 치즈까지 뿌려서 치즈길 만들면\n더 극락인 거 비밓,,>.<\n\n📍입분식가정집\n📍서울 송파구 오금로18길 10 2층\n📍매일 11:30-21:00\n\n맛집 콕 찝어서 알려줄게요! 신콕💖\n@sh.cookkook\n@sh.cookkook\n@sh.cookkook\n-\n-\n-\n#입분식가정집 #송파맛집 #송리단길 #송리단길맛집 #잠실맛집 #즉석떡볶이 #즉떡\n16시간', 'post_date': '2024-08-26 08:33:29', 'crawl_time': '2024-08-27 10:03:38', 'post_url': 'https://www.instagram.com/reel/C_IGZmRyUU9/?utm_source=ig_web_copy_link&igsh=MzRlODBiNWFlZA=='}


## 2. 인스타그램 특정 >> 계정 << 데이터 크롤링

##### [조건] 
#####  1. 로그인 O (비즈니스계정)
#####  2. 특정 계정 대상
#####  3. 계정명, 좋아요 수, 해시태그, 본문 벌크, 게시 날짜, 추출 날짜, 결과 csv 저장_(계정명.csv-추출)
##### [문제점]
#####  1. csv 파일로 떨궈지나, 데이터 추출이 안됌
#####  2. 이유를 모르겠음 ㅠㅠ

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
from datetime import datetime

# 데이터 저장을 위한 리스트 초기화
account_name_list = []
likes_list = []
hashtags_list = []
post_text_list = []
post_date_list = []
crawl_time_list = []
post_url_list = []

# 1. 웹드라이버 설정 및 인스타그램 로그인
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 인스타그램 로그인 페이지로 이동
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(5)  # 페이지 로드를 기다림

# 로그인 정보 입력
username = "snsdkf1234@naver.com"
password = "wel1234"

driver.find_element(By.NAME, "username").send_keys(username)
driver.find_element(By.NAME, "password").send_keys(password)
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button').click()
time.sleep(5)  # 로그인 후 대기

# 2. 특정 계정으로 이동
account_name = "seoul__nadri"  # 크롤링할 계정의 사용자 이름
driver.get(f"https://www.instagram.com/{account_name}/")
time.sleep(5)

# 게시물 URL 리스트 가져오기
post_elements = driver.find_elements(By.XPATH, '//article//a')
post_urls = [element.get_attribute('href') for element in post_elements]

# URL 출력 및 각 게시물에 대한 정보 추출
for url in post_urls:
    driver.get(url)
    time.sleep(5)  # 페이지 로드 대기

    # 계정명 추출
    try:
       account_name = driver.find_element(By.XPATH, '//a[contains(@class, "notranslate")]').text
    except:
       account_name = "N/A"

    # 좋아요 수 추출
    try:
        likes_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[contains(text(), "likes")]'))
        )
        likes = likes_element.text.split(' ')[0]  # 'likes' 텍스트 앞의 숫자만 추출
    except:
        likes = "N/A"
    
    # 해시태그 추출
    try:
        hashtags = [tag.text for tag in driver.find_elements(By.XPATH, '//a[contains(@href, "/explore/tags/")]')]
        hashtags = hashtags[:10]  # 상위 10개 해시태그만 가져오기
    except:
        hashtags = []
    
    # 본문 텍스트 추출
    try:
        post_text = driver.find_element(By.XPATH, '//div[@class="_a9zr"]').text
    except:
        post_text = ""
    
    # 게시 날짜 추출
    try:
        post_date_element = driver.find_element(By.XPATH, '//time')
        post_date = post_date_element.get_attribute('datetime')
        post_date = datetime.strptime(post_date, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")
    except:
        post_date = "N/A"
    
    # 현재 크롤링 시간
    crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 데이터 리스트에 추가
    account_name_list.append(account_name)
    likes_list.append(likes)
    hashtags_list.append(', '.join(hashtags))
    post_text_list.append(post_text)
    post_date_list.append(post_date)
    crawl_time_list.append(crawl_time)
    post_url_list.append(url)

    print(f"Crawled: {url}")

# 4. 데이터프레임 생성 및 CSV 저장
data = {
    'account_name': account_name_list,
    'likes': likes_list,
    'hashtags': hashtags_list,
    'post_text': post_text_list,
    'post_date': post_date_list,
    'crawl_time': crawl_time_list,
    'post_url': post_url_list
}

df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv(f'{account_name}.csv', index=False, encoding='utf-8-sig')

print(f"Data saved to {account_name}.csv")

# 드라이버 종료
driver.quit()


Data saved to seoul__nadri.csv
